In [ ]:
# Here Sir I am Importing necessary packages
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import model_from_json
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
from tensorflow.keras.layers import BatchNormalization, Conv2D, AveragePooling2D, MaxPooling2D
from tensorflow.keras.layers import ZeroPadding2D, Activation, Dense, Flatten, Input, add
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
import h5py

In [ ]:
# Here I am Loading and processing the A-Z dataset
def load_az_data(path):
    df = pd.read_csv(path, header=None)
    labels = df[0].values
    data = df.drop([0], axis=1).values
    data = data.reshape((-1, 28, 28))
    labels = labels.astype(np.uint8)
    data = data.astype(np.float32)
    return (data, labels)

In [ ]:
# Here I am Loadinging and process the MNIST 0-9 dataset
def load_mnist_data():
    (trainData, trainLabels), (testData, testLabels) = mnist.load_data()
    data = np.vstack([trainData, testData])
    labels = np.hstack([trainLabels, testLabels])
    return (data, labels)

In [ ]:
# Here I am Combining A-Z and MNIST datasets
def combine_sets(az_path):
    az_data, az_labels = load_az_data(az_path)
    mnist_data, mnist_labels = load_mnist_data()

    az_labels = az_labels + 10
    data = np.vstack([az_data, mnist_data])
    labels = np.hstack([az_labels, mnist_labels])

    data = [cv2.resize(img, (32, 32)) for img in data]
    data = np.array(data, dtype=np.float32)

    data = np.expand_dims(data, axis=-1)
    data = data / 255.0

    return (data, labels)

In [ ]:
# Resnet Implementation
from tensorflow.keras.layers import BatchNormalization, Conv2D, AveragePooling2D, Activation, Dense, Flatten, Input, add
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import SGD

class ResNet:
    @staticmethod
    def residual_module(data, K, stride, chanDim, red=False, reg=0.0001, bnEps=2e-5, bnMom=0.9):
        shortcut = data

        bn1 = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(data)
        act1 = Activation("relu")(bn1)
        conv1 = Conv2D(int(K * 0.25), (1, 1), use_bias=False, kernel_regularizer=l2(reg))(act1)

        bn2 = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(conv1)
        act2 = Activation("relu")(bn2)
        conv2 = Conv2D(int(K * 0.25), (3, 3), strides=stride, padding="same", use_bias=False, kernel_regularizer=l2(reg))(act2)

        bn3 = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(conv2)
        act3 = Activation("relu")(bn3)
        conv3 = Conv2D(K, (1, 1), use_bias=False, kernel_regularizer=l2(reg))(act3)

        if red:
            shortcut = Conv2D(K, (1, 1), strides=stride, use_bias=False, kernel_regularizer=l2(reg))(act1)

        x = add([conv3, shortcut])

        return x

    @staticmethod
    def build(width, height, depth, classes, stages, filters, reg=0.0001, bnEps=2e-5, bnMom=0.9):
        inputShape = (height, width, depth)
        chanDim = -1

        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            chanDim = 1

        inputs = Input(shape=inputShape)
        x = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(inputs)
        x = Conv2D(filters[0], (3, 3), use_bias=False, padding="same", kernel_regularizer=l2(reg))(x)

        for i in range(0, len(stages)):
            stride = (1, 1) if i == 0 else (2, 2)
            x = ResNet.residual_module(x, filters[i + 1], stride, chanDim, red=True, bnEps=bnEps, bnMom=bnMom)

            for j in range(0, stages[i] - 1):
                x = ResNet.residual_module(x, filters[i + 1], (1, 1), chanDim, bnEps=bnEps, bnMom=bnMom)

        x = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(x)
        x = Activation("relu")(x)
        x = AveragePooling2D((8, 8))(x)

        x = Flatten()(x)
        x = Dense(classes, kernel_regularizer=l2(reg))(x)
        x = Activation("softmax")(x)

        model = Model(inputs, x, name="resnet")

        return model

In [ ]:
# Sir the file is stored at this loaction(Only for alphabets) for digits I used mnsit dataset, no need to download as above function will do.

path = "C:\\Users\\DELL\\OneDrive\\New folder\\A_Z Handwritten Data.csv"


data, labels = combine_sets(path)

In [ ]:
# Here I Binarize the labels
le = LabelBinarizer()
labels = le.fit_transform(labels)
classTotals = labels.sum(axis=0)
classWeight = {}
for i in range(0, len(classTotals)):
    classWeight[i] = classTotals.max() / classTotals[i]

(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.20, stratify=labels, random_state=42)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data augmentation
aug = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.05,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    horizontal_flip=False,
    fill_mode="nearest"
)

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import SGD
# Initialize and compile the model
EPOCHS = 10
INIT_LR = 1e-1
BS = 128
# Learning rate schedule function
def lr_schedule(epoch, lr):
    return lr * 0.9  # Adjust this decay factor as needed

# Initialize the optimizer without the decay argument
opt = SGD(learning_rate=INIT_LR)

# Compile the model
model = ResNet.build(32, 32, 1, len(le.classes_), (3, 3, 3), 
                     (64, 64, 128, 256), reg=0.0005)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

# Define the learning rate scheduler callback
lr_scheduler = LearningRateScheduler(lr_schedule)

# Train the model with the learning rate scheduler
H = model.fit(
    aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY),
    steps_per_epoch=len(trainX) // BS,
    epochs=EPOCHS,
    class_weight=classWeight,
    callbacks=[lr_scheduler],
    verbose=1,
      
)

# Save model architecture and weights
model.save("handwriting_recognition_model.h5")

In [ ]:
# Evaluate the model
labelNames = "0123456789"
labelNames += "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
labelNames = [l for l in labelNames]

predictions = model.predict(testX, batch_size=BS)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=labelNames))

In [ ]:
plt.plot(H.history["loss"], label = "Train Loss")
plt.plot(H.history["val_loss"], label = "Validation Loss")

plt.title("Train Loss vs Validation Loss", size = 16)
plt.xlabel("Epochs")
plt.ylabel("Loss")

plt.legend()
plt.show()

In [ ]:
# Plot the training loss and accuracy
plt.plot(H.history["loss"], label="Train Loss")
plt.plot(H.history["val_loss"], label="Validation Loss")
plt.title("Train Loss vs Validation Loss", size=16)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
# Save the model to an HDF5 file
model.save("handwritten_recognition_model.h5")